In [1]:
from pyspark.sql import SparkSession
from sedona.spark import *
from pyspark.sql.functions import *
import time

config = SedonaContext.builder() .\
    config("spark.executor.memory", "26g") .\
    config("spark.driver.memory", "24g") .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.5_2.12:1.7.0,'
           'org.datasyslab:geotools-wrapper:1.7.0-28.5'). \
    getOrCreate()

sedona = SedonaContext.create(config)
# sedona.sparkContext.setLogLevel("OFF")
cores = sedona.sparkContext.defaultParallelism

25/01/24 13:56:25 WARN Utils: Your hostname, marvin resolves to a loopback address: 127.0.1.1; using 172.20.27.4 instead (on interface eth0)
25/01/24 13:56:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/data/homes_data/sudheer/benchmark_data/sedona_venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /data/homes_data/sudheer/.ivy2/cache
The jars for the packages stored in: /data/homes_data/sudheer/.ivy2/jars
org.apache.sedona#sedona-spark-shaded-3.5_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c1c99299-0529-44ed-9479-7c0a9ca91b9d;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-shaded-3.5_2.12;1.7.0 in central
	found org.datasyslab#geotools-wrapper;1.7.0-28.5 in central
:: resolution report :: resolve 202ms :: artifacts dl 9ms
	:: modules in use:
	org.apache.sedona#sedona-spark-shaded-3.5_2.12;1.7.0 from central in [default]
	org.datasyslab#geotools-wrapper;1.7.0-28.5 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------------------------------

Available layers: ['U2006_CLC2000_V2020_20u1']


In [2]:
# # Check layers in .gpkg
# import geopandas as gpd
# from fiona import listlayers

# path = "./data_Corine/land_cover_100m.gpkg"

# print("Available layers:", listlayers(path))

In [2]:
# Read files
# Everything in: EPSG:3035

lcover_path = "./data_Corine/land_cover_100m.parquet"
EU_cens_path = "./data_EU/census_grid_EU/grids.parquet"
italy_cens_path = "./data_Italy/merged/merged_pop_geom/merged.parquet"
germ_path = "./data_Germany/germ_grid.parquet"
contr_path = "./data_EU/countries_shp/"
comuni_path = "./data_EU/comuni_shp/"



# lcover_df = sedona.read.format("geoparquet").option("legacyMode", "true").load(lcover_path)
eu_df = sedona.read.format("geoparquet").option("legacyMode", "true").load(EU_cens_path)
# italy_df = sedona.read.format("geoparquet").option("legacyMode", "true").load(italy_cens_path)
# italy_df = italy_df.withColumn("geometry", expr("ST_MakeValid(geometry)"))
# germ_df = sedona.read.format("geoparquet").option("legacyMode", "true").load(germ_path)
# lcover_df = sedona.read.format("geopackage").option("tableName", "U2006_CLC2000_V2020_20u1").load(lcover_path)
# eu_df = sedona.read.format("geopackage").option("tableName", "census2021").load(EU_cens_path)
# italy_df = sedona.read.format("geopackage").option("tableName", "italy_census").load(italy_cens_path)
# germ_df = sedona.read.format("geopackage").option("tableName", "census_grid").load(germ_path)

contr_df = sedona.read.format("shapefile").load(contr_path)
comuni_df = sedona.read.format("shapefile").load(comuni_path)



In [3]:
# Column names
print(f"lcover_df: {lcover_df.columns}")
print()
print(f"comuni_df: {comuni_df.columns}")
print()
print(f"contr_df: {contr_df.columns}")
print()
print(f"germ_df: {germ_df.columns}")
print()
print(f"eu_df: {eu_df.columns}")
print()
print(f"italy_df: {italy_df.columns}")
print()

lcover_df: ['code_00', 'ID', 'Remark', 'Area_Ha', 'geometry']

comuni_df: ['geometry', 'COMM_ID', 'CNTR_ID', 'CNTR_CODE', 'COMM_NAME', 'NAME_ASCI', 'TRUE_FLAG', 'NSI_CODE', 'NAME_NSI', 'NAME_LATN', 'NUTS_CODE', 'FID']

contr_df: ['geometry', 'CNTR_ID', 'CNTR_NAME', 'NAME_ENGL', 'NAME_FREN', 'ISO3_CODE', 'SVRG_UN', 'CAPT', 'EU_STAT', 'EFTA_STAT', 'CC_STAT', 'NAME_GERM']

germ_df: ['GITTER_ID_100m', 'x_mp_100m', 'y_mp_100m', 'Einwohner', 'geometry']

eu_df: ['GRD_ID', 'T', 'M', 'F', 'Y_LT15', 'Y_1564', 'Y_GE65', 'EMP', 'NAT', 'EU_OTH', 'OTH', 'SAME', 'CHG_IN', 'CHG_OUT', 'LAND_SURFACE', 'POPULATED', 'CONFIDENTIALSTATUS', 'geometry']

italy_df: ['SEZ21_ID', 'POP21', 'FAM21', 'geometry']



In [3]:
# Join, Overlap geom, Intersection ratio and Export

from contextlib import contextmanager
from pyspark.storagelevel import StorageLevel

@contextmanager
def get_time(task_name):
    # print(f"{task_name}", end=" ", flush=True)
    start = time.time()
    yield
    elapsed = time.time() - start

    print(f"{task_name}... DONE in {(elapsed/60):.2f} min" \
          if elapsed >= 60 else f"{task_name}... DONE in {elapsed:.2f} sec")


def join_chey(df1, df2, pred= "ST_Intersects", create_geom=False, cache=False, export=False, name="unnamed", df2_grid=None):
    """
    Join, Overlap geom, Intersection ratio and Export
    
    """
    
    if not create_geom:
        res = df1.alias("a") \
            .join(df2.alias("b"), expr(f"{pred}(a.geometry, b.geometry)")) \
            .select(
                *[f"a.{c}" for c in df1.columns],
                *[f"b.{c}" for c in df2.columns if c != "geometry" and c not in df1.columns]
            )
        if export:
            with get_time(f"Exporting to ./outputs/{name}"):
                res = res.repartition(1)
                path = "./outputs"
                res.write.mode("overwrite").format("geoparquet").save(f"{path}/" + f"/{name}")
        return res
    
    else:
        if df2_grid is not None and df2_grid > 0:
            df_2_area = df2_grid**2
            res = df1.alias("a") \
                .join(df2.alias("b"), expr(f"{pred}(a.geometry, b.geometry)")) \
                .select(
                    expr("ST_Intersection(a.geometry, b.geometry)").alias("geometry"),
                    *[f"a.{c}" for c in df1.columns if c != "geometry"],
                    *[f"b.{c}" for c in df2.columns if c != "geometry"]
                ) \
                .withColumn("int_ratio", expr(f"ST_Area(geometry) / {df_2_area}"))

        else:
            res = df1.alias("a") \
                .join(df2.alias("b"), expr(f"{pred}(a.geometry, b.geometry)")) \
                .select(
                    expr("ST_Intersection(a.geometry, b.geometry)").alias("geometry"),
                    *[f"a.{c}" for c in df1.columns if c != "geometry"], 
                    expr("b.geometry").alias("b_geom"),
                    *[f"b.{c}" for c in df2.columns if c != "geometry"]
                ) \
                .withColumn("int_ratio", expr("ST_Area(geometry) / ST_Area(b_geom)")) \
                .drop("b_geom")
        
        if cache:
            if res.storageLevel != StorageLevel.NONE:
                res.unpersist()
            res.cache()
            if not export:
                res.count()
        
        if export:
            # res = res.repartition(1)
            path = "./outputs"
            res.write.mode("overwrite").format("geoparquet").save(f"{path}/" + f"/{name}")
        
        return res

In [3]:
contr_df.repartition(cores)
res = contr_df.alias("a").join(eu_df.alias("b"), expr("ST_Relate(a.geometry, b.geometry, 'T********')")).select("a.*", "b.*") 
print(res.count())
res.cache()

ERROR:root:KeyboardInterrupt while sending command.                (5 + 5) / 10]
Traceback (most recent call last):
  File "/data/homes_data/sudheer/benchmark_data/sedona_venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/homes_data/sudheer/benchmark_data/sedona_venv/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [6]:
print(res.count())

25/01/24 13:30:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
ERROR:root:KeyboardInterrupt while sending command.                (5 + 5) / 10]
Traceback (most recent call last):
  File "/data/homes_data/sudheer/benchmark_data/sedona_venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/homes_data/sudheer/benchmark_data/sedona_venv/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [5]:
# Basic Join

name = "eucom_eugrid"

comuni_df = comuni_df.repartition(cores)

with get_time("Joining EU com, EU grid"):

    res = join_chey(comuni_df, eu_df, pred="ST_Contains", create_geom=True, export=True, name=name)

    print(f"res size: {res.count():,}")


25/01/21 16:54:48 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.
25/01/21 16:59:23 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


res size: 3,017,518
Joining EU com, EU grid... DONE in 8.06 min


In [4]:
# Basic Join

name = "eucom_eugrid"

with get_time("Joining EU com, EU grid"):

    res = join_chey(comuni_df, eu_df, pred="ST_Contains", create_geom=False, export=False, name=name)

    print(f"res size: {res.count():,}")


25/01/21 12:22:56 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


res size: 3,017,518
Joining EU com, EU grid... DONE in 9.86 min


In [27]:
from pyspark.storagelevel import StorageLevel

if res.storageLevel == StorageLevel.NONE:
    print(res.storageLevel)


Serialized 1x Replicated


In [5]:
# Italy x EU grids

res = italy_df.alias("a").join(eu_df.alias("b"), expr("ST_Intersects(a.geometry, b.geometry)"))

name = "IT_EUgr"
start = time.time()
res.write.mode("overwrite").parquet(f"./outputs/{name}.parquet")
# print(f"Result: {res.count():,}")
end = time.time()

if end - start <= 60:
    print(f"Elapsed time: {(end-start):.2f} sec")
else:
    print(f"Elapsed time: {(end-start)/60:.2f} min")
    

25/01/17 15:48:22 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


Result: 1,905,529
Elapsed time: 1.08 min


In [7]:
# Italy x EU COMUNI

res = italy_df.alias("a").join(comuni_df.alias("b"), expr("ST_Intersects(a.geometry, b.geometry)"))

name = "IT_EUcom"
start = time.time()
res.write.mode("overwrite").parquet(f"./outputs/{name}.parquet")
# print(f"Result: {res.count():,}")
end = time.time()

if end - start <= 60:
    print(f"Elapsed time: {(end-start):.2f} sec")
else:
    print(f"Elapsed time: {(end-start)/60:.2f} min")
    

25/01/17 15:52:19 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


Result: 913,081
Elapsed time: 1.26 min


In [8]:
# Self join Italy

res = italy_df.alias("a").join(italy_df.alias("b"), expr("ST_Intersects(a.geometry, b.geometry)"))

start = time.time()
print(f"Result: {res.count():,}")
end = time.time()

if end - start <= 60:
    print(f"Elapsed time: {(end-start):.2f} sec")
else:
    print(f"Elapsed time: {(end-start)/60:.2f} min")
    

25/01/17 16:06:16 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


Result: 4,786,653
Elapsed time: 4.91 min


In [1]:
import geopandas as gpd
from contextlib import contextmanager
import time

@contextmanager
def get_time(task_name):
    print(f"{task_name}...", end="", flush=True)
    start = time.time()
    yield
    elapsed = time.time() - start
    if elapsed >= 60:
        print(f"\b DONE in {(elapsed/60):.2f} min")
    else:
        print(f"\b DONE in {elapsed:.2f} sec")

def join_geopandas(df1, df2, pred="intersects", create_geom=False, export=False, name="unnamed", output_path="./outputs"):
    """
    Join, Overlap geom, Intersection ratio and Export using GeoPandas
    """
    
    if pred == "intersects":
        res = gpd.sjoin(df1, df2, how="inner", predicate="intersects")
    elif pred == "contains":
        res = gpd.sjoin(df1, df2, how="inner", predicate="contains")
    elif pred == "within":
        res = gpd.sjoin(df1, df2, how="inner", predicate="within")
    else:
        raise ValueError(f"Unsupported predicate: {pred}")
    
    res = res[[*df1.columns, *[c for c in df2.columns if c != "geometry" and c not in df1.columns]]]
    if 'FID' in res.columns:
        res = res.drop(columns=['FID'])

    if create_geom:
        res['geometry'] = res.apply(lambda row: row['geometry'].intersection(row['geometry_right']), axis=1)
        res = res.drop(columns=['geometry_right'])


    if export:
        with get_time(f"Exporting to {output_path}/{name}.gpkg"):
            res.to_file(f"{output_path}/{name}.gpkg", driver="GPKG")
    
    return res



In [5]:
import geopandas as gpd
import pandas as pd

comuni_path = "./data_EU/comuni_shp/"
EU_cens_path = "./data_EU/census_grid_EU/grids.gpkg"

comuni_df = gpd.read_file(comuni_path)
eu_df = gpd.read_file(EU_cens_path)


name = "eucom_eugrid_GeoPandas"

with get_time("Joining EU com, EU grid"):
    res = join_geopandas(comuni_df, eu_df, pred="contains", export=True, name=name, output_path="./outputs")
    print(f"Result size: {len(res):,}")



Joining EU com, EU grid...Exporting to ./outputs/eucom_eugrid_GeoPandas.gpkg.. DONE in 7.03 min
Result size: 3,017,518
 DONE in 7.39 min


In [ ]:
# ================================================ GEOPANDAS ================================================ 

In [10]:
# Self Join Italy
# Geopandas

import pandas as pd
import geopandas as gpd
import time

a = gpd.read_file("./data_Italy/merged/merged_pop_geom/merged.gpkg")

start = time.time()
b = gpd.read_file("./data_Italy/merged/merged_pop_geom/merged.gpkg")

grids_enr = gpd.sjoin(a, b, how="left", predicate="intersects")
print(f"Result: {grids_enr.shape[0]:,}")
end = time.time()

if end - start <= 60:
    print(f"Elapsed time: {(end-start):.2f} sec")
else:
    print(f"Elapsed time: {(end-start)/60:.2f} min")

Result: 4,786,653
Elapsed time: 1.80 min


In [1]:
# EU Gr x Italy
# Geopandas

import pandas as pd
import geopandas as gpd
import time

start = time.time()
a = gpd.read_file("./data_Italy/merged/merged_pop_geom/merged.gpkg")
b = gpd.read_file("./data_EU/census_grid_EU/grids.gpkg")

grids_enr = gpd.sjoin(a, b, how="left", predicate="intersects")
print(f"Result: {grids_enr.shape[0]:,}")
end = time.time()

if end - start <= 60:
    print(f"Elapsed time: {(end-start):.2f} sec")
else:
    print(f"Elapsed time: {(end-start)/60:.2f} min")


Result: 1,905,542
Elapsed time: 2.71 min


In [11]:
# EU Gr x LandCover
# Geopandas

import pandas as pd
import geopandas as gpd
import time

start = time.time()
a = gpd.read_file("./data_Corine/land_cover_100m.gpkg")
b = gpd.read_file("./data_EU/census_grid_EU/grids.gpkg")

grids_enr = gpd.sjoin(a, b, how="left", predicate="intersects")
print(f"Result: {grids_enr.shape[0]:,}")
end = time.time()

if end - start <= 60:
    print(f"Elapsed time: {(end-start):.2f} sec")
else:
    print(f"Elapsed time: {(end-start)/60:.2f} min")


Result: 15,297,508
Elapsed time: 6.74 min


In [ ]:
# Comuni x LandCover
# Geopandas

import pandas as pd
import geopandas as gpd
import time

start = time.time()
a = gpd.read_file("./data_Corine/land_cover_100m.gpkg")
b = gpd.read_file("./data_EU/comuni_shp/")

grids_enr = gpd.sjoin(a, b, how="left", predicate="intersects")
print(f"Result: {grids_enr.shape[0]:,}")
end = time.time()

if end - start <= 60:
    print(f"Elapsed time: {(end-start):.2f} sec")
else:
    print(f"Elapsed time: {(end-start)/60:.2f} min")


In [2]:
# LandCover x LandCover
# Geopandas

import pandas as pd
import geopandas as gpd
import time

start = time.time()
a = gpd.read_file("./data_Corine/land_cover_100m.gpkg")
b = gpd.read_file("./data_Corine/land_cover_100m.gpkg")

grids_enr = gpd.sjoin(a, b, how="left", predicate="intersects")
print(f"Result: {grids_enr.shape[0]:,}")
end = time.time()

if end - start <= 60:
    print(f"Elapsed time: {(end-start):.2f} sec")
else:
    print(f"Elapsed time: {(end-start)/60:.2f} min")


Result: 12,804,697
Elapsed time: 432.87 min


In [ ]:
from sedona.sql import utils

In [ ]:
import geopandas as gpd
import time

s = time.time()
gdf1 = gpd.read_file("./data_EU/census_grid_EU/grids.gpkg", layer="census2021")
e = time.time()
if e - s <= 60:
    print(f"READ: {(e-s):.2f} sec")
else:
    print(f"READ: {(e-s)/60:.2f} min")
gdf2 = gpd.read_file("./data_EU/countries_shp/")

print(gdf1.shape[0])

s = time.time()
grids_enr = gpd.sjoin(gdf1, gdf2, how="inner", predicate="intersects")
e = time.time()
if e - s <= 60:
    print(f"JOIN: {(e-s):.2f} sec")
else:
    print(f"JOIN: {(e-s)/60:.2f} min")


In [ ]:
print(grids_enr.shape[0])

In [ ]:
import geopandas as gpd 

gdf1 = gpd.read_file("./data_Italy/merged/merged_pop_geom/")
print(gdf1.crs)
gdf1 = gdf1.to_crs("EPSG:3035")
print(gdf1.crs)
# Save to GeoPackage
gdf1.to_file("merged.gpkg", driver="GPKG", layer="italy_census")




In [ ]:
gdf.to_file("IT_census.geojson", driver="GeoJSON")